# Optically-dark short GRBs

## Imports/loading/setup

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, importlib
from astropy.io import votable, ascii
from scipy import interpolate
from custom_utils import AsymmetricUncertainty

alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
numeric = ".0123456789"
calc_code = importlib.import_module("Calculation Code.main")
graph_code = importlib.import_module("Graphing Code.Graphing_Beta_OX")

class custom_iter: # custom iterator class that allows for retrieval of current element w/out advancing
    def __init__(self, iterable):
        self.iterator = iter(iterable)
        self.current = None
    def __next__(self):
        try:
            self.current = next(self.iterator)
        except StopIteration:
            self.current = None
        finally:
            return self.current

In [ ]:
swift = pd.read_html("https://swift.gsfc.nasa.gov/archive/grb_table/fullview/")[0] # get latest Swift catalog
swift.columns = [col[0] for col in swift.columns] # reduce MultiIndex
sGRBs = swift[swift["BAT T90[sec]"].apply(pd.to_numeric, errors="coerce") <= 2] # filter catalog for GRBs with a valid T90 that is <2s

## Data parsing and cleanup

In [ ]:
# format columns
sGRBs.drop(['Time[UT]', 'BAT RA(J2000)', 'BAT Dec(J2000)', 'BAT 90%Error Radius[arcmin]',
            'BAT Fluence(15-150 keV)[10-7 erg/cm2]', 'BAT Fluence90% Error(15-150 keV)[10-7 erg/cm2]',
            'BAT 1-sec PeakPhoton Flux(15-150 keV)[ph/cm2/sec]', 'BAT 1-sec PeakPhoton Flux90% Error(15-150 keV)[ph/cm2/sec]',
            'BAT Photon Index(15-150 keV)(PL = simple power-law,CPL = cutoff power-law)', 'BAT Photon Index90% Error(15-150 keV)',
            'XRT RA(J2000)', 'XRT Dec(J2000)', 'XRT 90%Error Radius[arcsec]', 'XRT Column Density(NH)[1021 cm-2]',
            'XRT Early Flux(0.3-10 keV)[10-11 erg/cm2/s]', 'XRT 11 Hour Flux(0.3-10 keV)[10-11 erg/cm2/s]',
            'XRT 24 Hour Flux(0.3-10 keV)[10-11 erg/cm2/s]', 'UVOT RA(J2000)', 'UVOT Dec(J2000)', 'UVOT 90%Error Radius[arcsec]',
            'Host Galaxy', 'Comments', 'References', 'Burst Advocate'],axis=1,inplace=True) # already in master catalog
sGRBs.rename(columns={'XRT Time to FirstObservation[sec]':'XRT dt [sec]',
                      'UVOT Time toFirst Observation[sec]':'UVOT dt [sec]',
                      'UVOT Other FilterMagnitudes':'Other UVOT Filters'},inplace=True)

mags = pd.Series([]) # numerical magnitudes
lims = pd.Series([]) # what kind of limit?
for idx,mag in sGRBs["UVOT Magnitude"].iteritems():
    #print(idx)
    try:
        mags[idx] = float(mag)
        lims[idx] = "equal"
    except ValueError:
        if ">" in mag:
            lims[idx] = "brightest"
            mags[idx] = float("".join([char for char in mag.split()[0] if char in "0123456789."]))
        elif "=" in mag:
            lims[idx] = "equal"
            mags[idx] = float("".join([char for char in mag.split()[0] if char in "0123456789."]))
        else:
            print(mag)
            lims[idx] = "equal"
            mags[idx] = np.nan

sGRBs["UVOT Vmag"] = mags
sGRBs["UVOT Vmag lim"] = lims

In [ ]:
def split_filters(string):
    UVOT_filters = ["B","U","UVW1","UVM2","UVW2","White"]
    name_idxs = custom_iter([string.index(i) for i in UVOT_filters if i in string])
    split_list = [string[name_idxs.current:next(name_idxs)] for i in range(len(UVOT_filters))]
    split_list = [item for item in split_list if len(item)>0]
    return np.unique(split_list).tolist()

In [ ]:
optical_obs = []
for idx,entry in sGRBs["Other UVOT Filters"].iteritems():
    if pd.isna(entry):
        entries = []
    else:
        entries = split_filters(entry)
    try:
        filterdict = dict([entry.split(">") if ">" in entry else entry.split("=") if "=" in entry else None for entry in entries])
        filterdict["idx"] = idx
        optical_obs.append(filterdict)
    except:
        print("error",idx,end="; ")

for obs in optical_obs:
    sGRBs["Other UVOT Filters"][obs["idx"]] = obs

In [ ]:
sGRBs["Redshift"] = ["".join([char for char in entry if char in numeric]) if type(entry) is str else entry for entry in sGRBs["Redshift"]]

In [ ]:
sGRBs.to_csv("./Required Files/Files for Loading/Swift_sGRB_catalog.csv",index=False)
sGRBs.head()

## Legacy and new data imports

In [ ]:
BetaXData = pd.read_csv("./Required Files/Legacy Data/BetaXData.csv", header=None)
BetaXData.columns = ["GRB","BetaX","Beta_X_pos","Beta_X_neg"]
BetaXData["GRB"] = [entry.split("-")[-1] for entry in BetaXData["GRB"]]

OpticalData = pd.read_csv("./Required Files/Legacy Data/OpticalData.csv", header=None)
OpticalData.columns = ["GRB","Time","Observatory","Instrument","Filter","Exposure","F_o","e_F_o"]
OpticalData["GRB"] = [entry.split("-")[-1] for entry in OpticalData["GRB"]]
# OpticalData["dt"] = OpticalData["dt"]*60*60

XRayData = pd.read_csv("./Required Files/Legacy Data/XRayData.csv", header=None)
XRayData.columns = ["GRB","Time","Exposure","F_x","e_F_x"]

filters = pd.read_csv("Required Files/Legacy Data/FilterInfo.csv", header=None)
filters.columns = ["Observatory","Instrument","Filter","Wavelength","Frequency"]

new_sGRBs = sGRBs[[int(grb[:6]) > 150301 for grb in sGRBs["GRB"]]] # Fong et al. 2015 has data through March 2015, i.e. 150301A

In [ ]:
new_sGRBs

In [ ]:
new_optical = pd.read_excel("./Required Files/Files for Loading/newData.xlsx")

for col in ["GRB","TriggerNumber","Observatory","Instrument","Source","E(B-V)"]:
    for i in new_optical.index:
        if pd.isna(new_optical.loc[i,col]):
            new_optical.loc[i,col] = new_optical.loc[i-1,col] # deal with merged Excel cells

## Retrieve Swift XRT light curve and spectrum

In [ ]:
def XRT_lightcurve(burst_id):
    trigger = swift.loc[swift["GRB"] == burst, "TriggerNumber"]
    lightcurveURL = f"https://www.swift.ac.uk/xrt_curves/{int(trigger):0>8}/flux.qdp"
    
    fluxdata = pd.read_table(lightcurveURL, header=11).apply(pd.to_numeric, errors="coerce").dropna().reset_index(drop=True)
    fluxdata.columns = ["Time","Tpos","Tneg","Flux","Fluxpos","Fluxneg"]
    
    return fluxdata


def get_BetaX(burst_id):
    trigger = swift.loc[swift["GRB"] == burst, "TriggerNumber"]
    spectrumURL = f"https://www.swift.ac.uk/xrt_spectra/{int(trigger):0>8}/"
    
    spectra_tables = pd.read_html(spectrumURL)
    PC_table = spectra_tables[len(spectra_tables)-2]
    photon_index = PC_table.loc[PC_table[0]=="Photon index",1].values
    (Gamma, Gammapos, Gammaneg) = (float(num) for num in "".join([char for char in str(photon_index[0]) if char not in "[]()+-,"]).split())
    
    return Gamma, Gammapos, Gammaneg

In [ ]:
burst = "200411A"
fluxdata = XRT_lightcurve(burst)
Gamma, Gammapos, Gammaneg = get_BetaX(burst)

In [ ]:
plt.errorbar(fluxdata.Time,fluxdata.Flux,xerr=np.array(fluxdata.Tneg,fluxdata.Tpos).T,yerr=np.array(fluxdata.Fluxneg,fluxdata.Fluxpos).T,linestyle="",capthick=0)
plt.xscale("log")
plt.yscale("log")
plt.title("Swift XRT Lightcurve for GRB %s\n$\\beta_X = %.2f_{-%.2f}^{+%.2f}$" % (burst,Gamma-1.,Gammaneg,Gammapos))
plt.xlabel("Time since trigger [s]")
plt.ylabel("Flux (0.3-10 keV) [erg/s/cm$^2$]")
plt.grid(linestyle="--")
plt.xlim(3e1,1e5)
plt.show()

In [ ]:
for i in new_sGRBs.index:
    GRB_ID = new_sGRBs.loc[i,"GRB"]
    
    try:
        (Gamma, Gammapos, Gammaneg) = get_BetaX(GRB_ID)
        fluxdata = XRT_lightcurve(GRB_ID)
        
        new_sGRBs.loc[i,"Beta_X"] = 1. - Gamma
        new_sGRBs.loc[i,"Beta_X_neg"] = Gammapos
        new_sGRBs.loc[i,"Beta_X_pos"] = Gammaneg
        #print(f"Found {GRB_ID} at index {i}: {Gamma, Gammapos, Gammaneg}")
        
    except:
        print("Failed to retrieve info for GRB",GRB_ID)

## Correcting for extinction

$ A_b = R_b \cdot E_{B-V} $ for an arbitrary band $b$

In [ ]:
RbTable = pd.read_csv("./Required Files/Rb.csv") # Table 6 from Schlafly & Finkbeiner (2011)
Rb = interpolate.interp1d(RbTable["lambda_eff"],RbTable["R_b"]) # function that takes a wavelength [Ang] and returns the corresponding R_b value

In [ ]:
for i in new_optical.index:
    try:
        new_optical.loc[i,"Extinction"] = Rb(new_optical.loc[i,"λ_eff"])*new_optical.loc[i,"E(B-V)"]
    except ValueError:
        pass
        #print(new_optical.loc[i,"λ_eff"])

## Putting it all together

In [ ]:
#testcase = calc_code.GRB(burst, )
new_optical[~new_optical["Mag error"].apply(pd.to_numeric, errors="coerce").isna()]

In [ ]:
for i in testcase.index:
    optical_obs = testcase.loc[i,:]
    closest_xray = pd.DataFrame({"Time":[np.inf]})
    for j in fluxdata.index:
        xray_obs = fluxdata.loc[j,:]
        if np.abs(float(xray_obs["Time"]) - float(optical_obs["Time (s)"])) < float(closest_xray["Time"]):
            closest_xray = xray_obs
    print(optical_obs)
    print(closest_xray)
    print("dt",np.abs(float(xray_obs["Time"]) - float(optical_obs["Time (s)"])))
    #print(calc_code.GRB(datapoint["GRB"],))